In [57]:
import keras
import tensorflow as tf
import random
import shutil
import os
import re
keras.__version__

'2.6.0'

## Selecting species subset

In [2]:
dir_train_mini = "/courses/EDS232/inaturalist-2021/train_mini"

In [3]:
dirs_spp = os.listdir(dir_train_mini)
n_spp = len(dirs_spp)
n_spp

10000

In [4]:
seed_value = 71993

In [5]:
random.seed(seed_value)
i10 = random.sample(range(1, n_spp), 10)
i10

[6265, 8377, 9422, 2010, 7951, 3269, 4966, 3475, 9868, 6903]

In [6]:
for i in range(0, len(i10)):
    print(os.path.basename(dirs_spp[i10[i]]))

02163_Animalia_Arthropoda_Insecta_Lepidoptera_Saturniidae_Hyalophora_columbia
09827_Plantae_Tracheophyta_Pinopsida_Pinales_Pinaceae_Pinus_monophylla
00004_Animalia_Arthropoda_Arachnida_Araneae_Agelenidae_Eratigena_duellica
03353_Animalia_Chordata_Aves_Charadriiformes_Charadriidae_Charadrius_leschenaultii
00283_Animalia_Arthropoda_Insecta_Coleoptera_Chrysomelidae_Disonycha_glabrata
09526_Plantae_Tracheophyta_Magnoliopsida_Santalales_Santalaceae_Phoradendron_leucarpum
06969_Plantae_Tracheophyta_Magnoliopsida_Asterales_Asteraceae_Senecio_ampullaceus
04410_Animalia_Chordata_Aves_Pelecaniformes_Threskiornithidae_Theristicus_caudatus
03042_Animalia_Chordata_Amphibia_Anura_Ranidae_Rana_cascadae
04087_Animalia_Chordata_Aves_Passeriformes_Parulidae_Setophaga_townsendi


In [7]:
i2 = i10[0:2]

In [8]:
for i in range(0, len(i2)):
    print(os.path.basename(dirs_spp[i2[i]]))

02163_Animalia_Arthropoda_Insecta_Lepidoptera_Saturniidae_Hyalophora_columbia
09827_Plantae_Tracheophyta_Pinopsida_Pinales_Pinaceae_Pinus_monophylla


## Building file structure

In [10]:
original_dataset_dir = dir_train_mini

# The directory where we stored our smaller dataset
base_dir = "/Users/petermenzies/eds232-ml/inat-neural-nets/inat_subset"
os.mkdir(base_dir)

In [141]:
spp_names = []

for i in range(0, len(i10)):
    full_name = os.path.basename(dirs_spp[i10[i]]).lower()
    name_split = full_name.split('_')
    name = name_split[-2] + '_' + name_split[-1]
    spp_names.append(name)

In [172]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_dir_names = []
test_dir_names = []
validation_dir_names = []

for i in range(0, len(i10)):
    locals()['train_' + spp_names[i] + '_dir'] = os.path.join(train_dir, spp_names[i])
    locals()['validation_' + spp_names[i] + '_dir'] = os.path.join(validation_dir, spp_names[i])
    locals()['test_' + spp_names[i] + '_dir'] = os.path.join(test_dir, spp_names[i])
    
    train_dir_names.extend(['train_' + spp_names[i] + '_dir'])
    validation_dir_names.extend(['validation_' + spp_names[i] + '_dir'])
    test_dir_names.extend(['test_' + spp_names[i] + '_dir'])

In [175]:
# Creating directories

os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)

for i in train_dir_names:
    os.mkdir(locals()[i])

for i in validation_dir_names:
    os.mkdir(locals()[i])
    
for i in test_dir_names:
    os.mkdir(locals()[i])

In [177]:
# Copying subset of images from source directory to new structured directories

# Copying training images
for i in range(1, len(i10)):
    path = os.path.join(original_dataset_dir, dirs_spp[i10[i]])
    fnames = os.listdir(path)[0:30]
    
    for fname in fnames:
        src = os.path.join(original_dataset_dir, path, fname)
        dst = os.path.join(locals()[train_dir_names[i]], fname)
        shutil.copyfile(src, dst)
    
# Copying validation images
for i in range(1, len(i10)):
    path = os.path.join(original_dataset_dir, dirs_spp[i10[i]])
    fnames = os.listdir(path)[30:40]
    
    for fname in fnames:
        src = os.path.join(original_dataset_dir, path, fname)
        dst = os.path.join(locals()[validation_dir_names[i]], fname)
        shutil.copyfile(src, dst)
    
# Copying test images
for i in range(1, len(i10)):
    path = os.path.join(original_dataset_dir, dirs_spp[i10[i]])
    fnames = os.listdir(path)[40:50]
    
    for fname in fnames:
        src = os.path.join(original_dataset_dir, path, fname)
        dst = os.path.join(locals()[test_dir_names[i]], fname)
        shutil.copyfile(src, dst)

## Artificial neural net for binary species classification

In [59]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=10,
        class_mode='binary')

Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [54]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (10, 150, 150, 3)
labels batch shape: (10,)
